In [2]:
!pip install tensorflow

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/anaconda/envs/azureml_py38/lib/python3.10/site-packages/numpy-1.23.5.dist-info/METADATA'



In [ ]:
!pip install rdkit
import pandas as pd
import numpy as np
import tensorflow as tf
import rdkit
import seaborn as sn
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn import tree, model_selection
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV,\
RandomizedSearchCV
import sklearn
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report, matthews_corrcoef
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict

In [68]:
df = pd.read_csv("balanced_smiles_data_1.csv")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,Activity
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [69]:
Xfeatures = df.iloc[:, 0:2048]  # Selects columns 0 to 2047 (2048 columns in total)
Xfeatures.shape

(56742, 2048)

In [70]:
Yfeatures = df['Activity']
Yfeatures.shape

(56742,)

In [71]:
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,Yfeatures,test_size=0.3,random_state=42)

In [72]:
X_train.shape

(39719, 2048)

In [73]:
X_test.shape

(17023, 2048)

In [74]:
y_train.shape

(39719,)

In [75]:
y_test.shape

(17023,)

In [76]:
models= [tree.DecisionTreeClassifier()]
for model in models:
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    print(type(model).__name__)
    print(accuracy_score(y_test, prediction))
    print("Recall:",metrics.recall_score(y_test, prediction))
    print('matthew;s correlation coefficient',matthews_corrcoef(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print(classification_report(y_test, prediction))
    #validation_score = cross_val_score(model,Xfeatures,Yfeatures, cv=10)
    #print('Cross_validated score: ' , validation_score)
    #print('Cross_validated mean: ', validation_score.mean())


DecisionTreeClassifier
0.9054220760148035
Recall: 0.8522928209359897
matthew;s correlation coefficient 0.815308699252647
[[8183  357]
 [1253 7230]]
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      8540
           1       0.95      0.85      0.90      8483

    accuracy                           0.91     17023
   macro avg       0.91      0.91      0.91     17023
weighted avg       0.91      0.91      0.91     17023



In [77]:
dt_min_leaf_entropy = tree.DecisionTreeClassifier(max_depth=30,min_samples_leaf=20, random_state=42, criterion="entropy")
dt_min_leaf_entropy.fit(X_train, y_train)
y_test_pred = dt_min_leaf_entropy.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print("Recall:",metrics.recall_score(y_test, y_test_pred))
print("MCC",matthews_corrcoef(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

0.8252951888621277
Recall: 0.7867499705292939
MCC 0.6523689221791608
[[7375 1165]
 [1809 6674]]
              precision    recall  f1-score   support

           0       0.80      0.86      0.83      8540
           1       0.85      0.79      0.82      8483

    accuracy                           0.83     17023
   macro avg       0.83      0.83      0.82     17023
weighted avg       0.83      0.83      0.83     17023



In [78]:
models= [KNeighborsClassifier()]
for model in models:
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    print(type(model).__name__)
    print(accuracy_score(y_test, prediction))
    print("Recall:",metrics.recall_score(y_test, prediction))
    print('matthew;s correlation coefficient',matthews_corrcoef(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print(classification_report(y_test, prediction))
    #validation_score = cross_val_score(model,Xfeatures,Yfeatures, cv=10)
    #print('Cross_validated score: ' , validation_score)
    #print('Cross_validated mean: ', validation_score.mean())

KNeighborsClassifier
0.8711742936027728
Recall: 0.8159849109984675
matthew;s correlation coefficient 0.7467159917989619
[[7908  632]
 [1561 6922]]
              precision    recall  f1-score   support

           0       0.84      0.93      0.88      8540
           1       0.92      0.82      0.86      8483

    accuracy                           0.87     17023
   macro avg       0.88      0.87      0.87     17023
weighted avg       0.88      0.87      0.87     17023



In [ ]:
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
grid_search = GridSearchCV(KNeighborsClassifier(),
                           param_grid=grid_params)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

In [ ]:
models= [KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='distance')]
for model in models:
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    print(type(model).__name__)
    print(accuracy_score(y_test, prediction))
    print("Recall:",metrics.recall_score(y_test, prediction))
    print('matthew;s correlation coefficient',matthews_corrcoef(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    print(classification_report(y_test, prediction))
    #validation_score = cross_val_score(model,Xfeatures,Yfeatures, cv=10)
    #print('Cross_validated score: ' , validation_score)
    #print('Cross_validated mean: ', validation_score.mean())

In [80]:
rf_clf=RandomForestClassifier(random_state=42)
rf_clf.fit(X_train,y_train)
y_pred=rf_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print('matthew;s correlation coefficient',matthews_corrcoef(y_test, y_pred))
print(classification_report(y_test, y_pred))
#validation_score = cross_val_score(model,Xfeatures,Yfeatures, cv=10)
#print('Cross_validated score: ' , validation_score)
#print('Cross_validated mean: ', validation_score.mean())


0.9127063384832286
Recall: 0.8746905575857598
[[8117  423]
 [1063 7420]]
matthew;s correlation coefficient 0.8277111983000738
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      8540
           1       0.95      0.87      0.91      8483

    accuracy                           0.91     17023
   macro avg       0.92      0.91      0.91     17023
weighted avg       0.92      0.91      0.91     17023



In [ ]:
from sklearn.model_selection import GridSearchCV,\
RandomizedSearchCV
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}
grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

In [ ]:
model_grid = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='sqrt',
                       max_leaf_nodes=9, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=25,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
model_grid.fit(X_train, y_train)
y_pred_grid = model.predict(X_test)
print(accuracy_score(y_test, y_pred_grid))
print("Recall:",metrics.recall_score(y_test, y_pred_grid))
print('matthew;s correlation coefficient',matthews_corrcoef(y_test, y_pred_grid))
print(confusion_matrix(y_test, y_pred_grid))
print(classification_report(y_pred_grid, y_test))

In [ ]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Oerformance Evaluation
print(accuracy_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))
print('matthew;s correlation coefficient',matthews_corrcoef(y_test, y_pred))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))
validation_score = cross_val_score(model,Xfeatures,Yfeatures, cv=10)
print('Cross_validated score: ' , validation_score)
print('Cross_validated mean: ', validation_score.mean())

In [ ]:
param_grid= {'kernel': ('linear', 'rbf'),
...              'C': [1, 10, 100]}
grid_search = GridSearchCV(svm.SVC(),
                           param_grid=param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
#Create a svm Classifier
clf = svm.SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Oerformance Evaluation
print(accuracy_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))
print('matthew;s correlation coefficient',matthews_corrcoef(y_test, y_pred))
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))
#validation_score = cross_val_score(model,Xfeatures,Yfeatures, cv=10)
#print('Cross_validated score: ' , validation_score)
#print('Cross_validated mean: ', validation_score.mean())

In [ ]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
# define and fit the model
def get_model(X_train, y_train):
 # define model
 model = Sequential()
 model.add(Dense(100, input_shape=(2048,), activation='relu'))
 model.add(Dense(1, activation='relu'))
 # compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 # fit model
 model.fit(X_train, y_train, epochs=300, verbose=0)
 return model

# generate data
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,Yfeatures,test_size=0.3,random_state=42)

# fit model
model_ann = get_model(X_train, y_train)


# predict probabilities for test set
yhat_probs = model_ann.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = (model_ann.predict(X_test) > 0.5).astype("int32")
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
print("MCC:",matthews_corrcoef(y_test, yhat_classes))
# precision tp / (tp + fp)
precision = metrics.precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = metrics.recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = metrics.f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)
# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)

In [ ]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
# define and fit the model
def get_model(X_train, y_train):
 # define model
 model = Sequential()
 model.add(Dense(100, input_shape=(2048,), activation='sigmoid'))
 model.add(Dense(1, activation='sigmoid'))
 # compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 # fit model
 model.fit(X_train, y_train, epochs=300, verbose=0)
 return model

# generate data
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,Yfeatures,test_size=0.3,random_state=42)

# fit model
model_ann = get_model(X_train, y_train)


# predict probabilities for test set
yhat_probs = model_ann.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = (model_ann.predict(X_test) > 0.5).astype("int32")
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
print("MCC:",matthews_corrcoef(y_test, yhat_classes))
# precision tp / (tp + fp)
precision = metrics.precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = metrics.recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = metrics.f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)
# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)

In [ ]:
res = pd.DataFrame(y_test)
res.index = x_test.index # its important for comparison
res.columns = ["actual output"]
res.to_csv("Actual output.csv")
res = pd.DataFrame(prediction)
res.index = x_test.index # its important for comparison
res.columns = ["prediction"]
res.to_csv("prediction_results.csv")